In [6]:
import numpy as np
import pandas as pd
import datetime

In [2]:
import zipfile

train_zip = zipfile.ZipFile('../data/ECommAI_EUIR_round1_train_20190701.zip')
test_zip = zipfile.ZipFile('../data/ECommAI_EUIR_round2_train_20190821.zip')

In [4]:
print(train_zip.namelist())
print(test_zip.namelist())

['ECommAI_EUIR_round1_train_20190701/', 'ECommAI_EUIR_round1_train_20190701/item.csv', 'ECommAI_EUIR_round1_train_20190701/user.csv', 'ECommAI_EUIR_round1_train_20190701/user_behavior.csv']
['ECommAI_EUIR_round2_train_20190816/', 'ECommAI_EUIR_round2_train_20190816/user_behavior.csv', 'ECommAI_EUIR_round2_train_20190816/item.csv', 'ECommAI_EUIR_round2_train_20190816/user.csv']


In [8]:
df_user_train = pd.read_csv(train_zip.open('ECommAI_EUIR_round1_train_20190701/user.csv'),header=None)
df_user_train.columns = ['userID','gender','age','purchaseLevel']
df_item_train = pd.read_csv(train_zip.open('ECommAI_EUIR_round1_train_20190701/item.csv'),header=None)
df_item_train.columns = ['itemID','categoryID','shopID','brandID']
#df_log_train = pd.read_csv('datalab/35680/user_behavior_train.csv',header=None)
#df_log_train.columns = ['userID','itemID','behavior','timestap']


df_user_test = pd.read_csv(test_zip.open('ECommAI_EUIR_round2_train_20190816/user.csv'),header=None)
df_user_test.columns = ['userID','gender','age','purchaseLevel']

df_item_test = pd.read_csv(test_zip.open('ECommAI_EUIR_round2_train_20190816/item.csv'),header=None)
df_item_test.columns = ['itemID','categoryID','shopID','brandID']

df_log_test = pd.read_csv(test_zip.open('ECommAI_EUIR_round2_train_20190816/user_behavior.csv'),header=None)
df_log_test.columns = ['userID','itemID','behavior','timestap']

In [ ]:
测试集中有5万用户。一些同学不明白指定用户为测试集的意义。实际上，在真实的推荐场景下，为了测试方案效果，会随机选取一些用户测试新方案，而剩余用户仍采用旧方案。最后，对比点击率等指标。

用户的浏览（pv）、收藏（fav）、加购物车（cart）和购买（buy）的商品均认为是用户感兴趣的商品。
一种比较自然的推荐方案是，给用户推荐最近几天感兴趣的商品。若推荐列表长度不足50，则可以用流行度高的商品填补；若推荐列表长度超过50，则可依据感兴趣程度排序。

## Step 1. 基于用户行为日志统计item出现次数，从而获得受欢迎的商品Top50，并过滤掉大量低频item。

In [9]:
df_item = pd.concat([df_item_train,df_item_test])
df_item.drop_duplicates(inplace=True)
df_item.reset_index(drop=True,inplace=True)

In [10]:
#df_log = pd.concat([df_log_train,df_log_test])
df_log = df_log_test.copy()
df_log.reset_index(drop=True,inplace=True)

In [11]:
item_statistc = df_log.groupby(['itemID'])[['userID']].count()
item_statistc.reset_index(inplace=True)
item_statistc.columns = ['itemID','itemCount']

In [12]:
df_item = pd.merge(df_item,item_statistc)
df_item_selected = df_item[df_item['itemCount']>1]
df_item_selected.reset_index(drop=True,inplace=True)

In [13]:
df_item_sorted = df_item_selected.sort_values(by=['itemCount'],ascending=False)
df_item_sorted.reset_index(drop=True,inplace=True)

In [15]:
df_log.head()

,userID,itemID,behavior,timestap
0,286924,13546315,pv,1124981
1,893324,19447070,cart,292579
2,893324,19447070,pv,296258
3,153813,1251077,pv,243310
4,1403217,19735607,pv,476795


## Step 2. 依据近三天的日志为测试用户推荐，并进行填补等。

In [19]:
df_log['timestap'] = df_log['timestap'].apply(lambda x : datetime.datetime(2019,8,12) + datetime.timedelta(seconds=x))
df_log.head()

MemoryError: 

In [ ]:
df_log['day'] = df_log['date'].dt.day
df_log['weekday'] = df_log['date'].dt.weekday + 1

In [ ]:
df_log_selected = df_log[df_log['day']>=17]

In [ ]:
df_log_selected.loc[df_log_selected['behavior']=='pv','behavior'] = 1
df_log_selected.loc[df_log_selected['behavior']=='fav','behavior'] = 2
df_log_selected.loc[df_log_selected['behavior']=='cart','behavior'] = 3
df_log_selected.loc[df_log_selected['behavior']=='buy','behavior'] = 4

In [ ]:
item_Top50 = list( df_item_sorted.loc[:49,'itemID'] )

## 为测试集中用户做推荐。

In [ ]:
file = open('../data/submission_round2.csv','w')


In [ ]:
count = 0

for each_user in df_user_test['userID']:

    count = count + 1
    
    if count%500==0:
        print(count)
    
    df_tmp = df_log_selected[df_log_selected['userID']==each_user]
    df_tmp.reset_index(inplace=True,drop=True)
    
    itemListTmp = []
    
    if len(df_tmp) > 0:
        
        item_sta = df_tmp.groupby(['itemID'])['behavior'].sum()
        item_sta = item_sta.reset_index()
        item_sta_sorted = item_sta.sort_values(by=['behavior'],ascending=False)
        item_sta_sorted.reset_index(inplace=True,drop=True)
        
        itemListTmp = itemListTmp + list(item_sta_sorted.loc[:49,'itemID'])
        
        if len(itemListTmp) < 50:
            
            itemListTmp = itemListTmp + item_Top50[:(50-len(itemListTmp))]
        
    else:
        
        itemListTmp = item_Top50.copy()
    
    strTmp = str(each_user) + ',' + ','.join(map(lambda x:str(x),itemListTmp))
    
    file.write(strTmp+'\n')

file.close()